In [1]:
pip install tesseract

  Using cached tesseract-0.1.3.tar.gz (45.6 MB)Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started



You should consider upgrading via the 'C:\Users\masoud.sham\Anaconda3\python.exe -m pip install --upgrade pip' command.


  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562571 sha256=281bf2b342dc0d9f6c51810b678a1da253c1c32b101fec7765b935253b75b09b
  Stored in directory: c:\users\masoud.sham\appdata\local\pip\cache\wheels\7f\a1\69\fabe07004553a36d818e4657fed410daf525fe1ae161f469d3
Successfully built tesseract


In [3]:
pip install pytesseract

  Using cached pytesseract-0.3.8.tar.gz (14 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pytesseract: filename=pytesseract-0.3.8-py2.py3-none-any.whl size=14070 sha256=32085ec492621e42f52f37c3e98aba739fd70a785dc9dac7c778aaf7ab33aad4
  Stored in directory: c:\users\masoud.sham\appdata\local\pip\cache\wheels\ed\8a\c6\40c7ec06c2dd3df636832537238128a1471bf9c1b6a3a9bf40
Successfully built pytesseract
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\masoud.sham\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [236]:
import cv2
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\masoud.sham\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

In [237]:
def pre_process(img):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(grey, (5, 5), 1)

    edge = cv2.Canny(blur, 135, 200)

    kernal = np.ones((5, 5))

    dilate = cv2.dilate(edge, kernal, iterations=2)

    threshold = cv2.erode(dilate, kernal, iterations=1)

    return threshold

In [238]:
def get_contours(img):
    biggest = np.array([])
    max_area = 0
    contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)

        if area > 500:
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area

    points = cv2.drawContours(img_contour, biggest, -1, (255, 0, 0), 20)
    return biggest

In [239]:
def reshape(points):
    points = points.reshape((4, 2))
    new_points = np.zeros((4, 1, 2), np.int32)

    add = points.sum(1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    diff = np.diff(points, axis=1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]

    return new_points

In [240]:
def fix_image(img, contour):
    contour = reshape(contour)

    pts1 = np.float32(contour)
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    # get the warp perspective transform
    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    # apply to the image
    final_img = cv2.warpPerspective(img, matrix, (width, height))

    #crop the image
    cropped = final_img[20:final_img.shape[0] - 20, 20:final_img.shape[1] - 20]

    return cropped

In [291]:
file_name = r"E:\MTN Data Science Academy 2021\Tests and other files\Platinum Test\Capstone Student Material\Brilliant AC.jpg"

img = cv2.imread(file_name)
print(img.shape)
img_contour = img.copy()



scale_percent = 100
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

threshold = pre_process(img)
get_contour = get_contours(threshold)
fixed_image = fix_image(img, get_contour)

# extract text from image using tesseract

text = pytesseract.image_to_string(img)
print('text detected: \n' + text)

cv2.imshow('img', img)
cv2.imshow('fixed_image', fixed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


(2592, 4608, 3)
text detected: 
scope of Work

Brilliant Air Conditioners PTY LTD.
03 Feb 2022.
The work to be carried out will consist of the following items:

1. John Doe will provide maintenance work on the compressors of the air conditioning units.
2. The work will be done on the air conditioning units in the Johannesburg office.
3. The work will be carried out between 15 February 2022 and 2022/03/05.

In the event of any queries, please contact Sophie Williams on 082 221 1549.

Or on sophiew@brilliantac.com

Terms and Conditions.

Any item not mentioned in the scope of work will not be carried out.

Work will only take place once the payment is made in full, or if there is a service contract in place.
Work will be guaranteed for 3 months after the completion date.

Delays not caused by us will be chargeable.




In [403]:
text

'scope of Work\n\nBrilliant Air Conditioners PTY LTD.\n03 Feb 2022.\nThe work to be carried out will consist of the following items:\n\n1. John Doe will provide maintenance work on the compressors of the air conditioning units.\n2. The work will be done on the air conditioning units in the Johannesburg office.\n3. The work will be carried out between 15 February 2022 and 2022/03/05.\n\nIn the event of any queries, please contact Sophie Williams on 082 221 1549.\n\nOr on sophiew@brilliantac.com\n\nTerms and Conditions.\n\nAny item not mentioned in the scope of work will not be carried out.\n\nWork will only take place once the payment is made in full, or if there is a service contract in place.\nWork will be guaranteed for 3 months after the completion date.\n\nDelays not caused by us will be chargeable.\n\n'

In [404]:
import spacy
nlp = spacy.load('en_core_web_sm') 
doc  = nlp(text)

In [405]:
doc

scope of Work

Brilliant Air Conditioners PTY LTD.
03 Feb 2022.
The work to be carried out will consist of the following items:

1. John Doe will provide maintenance work on the compressors of the air conditioning units.
2. The work will be done on the air conditioning units in the Johannesburg office.
3. The work will be carried out between 15 February 2022 and 2022/03/05.

In the event of any queries, please contact Sophie Williams on 082 221 1549.

Or on sophiew@brilliantac.com

Terms and Conditions.

Any item not mentioned in the scope of work will not be carried out.

Work will only take place once the payment is made in full, or if there is a service contract in place.
Work will be guaranteed for 3 months after the completion date.

Delays not caused by us will be chargeable.


In [406]:
for token in doc:
    print(token)


scope
of
Work



Brilliant
Air
Conditioners
PTY
LTD
.


03
Feb
2022
.


The
work
to
be
carried
out
will
consist
of
the
following
items
:



1
.
John
Doe
will
provide
maintenance
work
on
the
compressors
of
the
air
conditioning
units
.


2
.
The
work
will
be
done
on
the
air
conditioning
units
in
the
Johannesburg
office
.


3
.
The
work
will
be
carried
out
between
15
February
2022
and
2022/03/05
.



In
the
event
of
any
queries
,
please
contact
Sophie
Williams
on
082
221
1549
.



Or
on
sophiew@brilliantac.com



Terms
and
Conditions
.



Any
item
not
mentioned
in
the
scope
of
work
will
not
be
carried
out
.



Work
will
only
take
place
once
the
payment
is
made
in
full
,
or
if
there
is
a
service
contract
in
place
.


Work
will
be
guaranteed
for
3
months
after
the
completion
date
.



Delays
not
caused
by
us
will
be
chargeable
.





In [407]:
for sent in doc.sents:
    print(sent)

scope of Work

Brilliant Air Conditioners PTY LTD.

03
Feb 2022.

The work to be carried out will consist of the following items:

1.
John Doe will provide maintenance work on the compressors of the air conditioning units.

2.
The work will be done on the air conditioning units in the Johannesburg office.

3.
The work will be carried out between 15 February 2022 and 2022/03/05.


In the event of any queries, please contact Sophie Williams on 082 221 1549.


Or on sophiew@brilliantac.com

Terms and Conditions.


Any item not mentioned in the scope of work will not be carried out.


Work will only take place once the payment is made in full, or if there is a service contract in place.

Work will be guaranteed for 3 months after the completion date.


Delays not caused by us will be chargeable.





In [408]:
for entity in doc.ents:
    print(f"{entity.text:>35} {entity.label_}")

                                 03 CARDINAL
                           Feb 2022 DATE
                                  1 CARDINAL
                           John Doe PERSON
                                  2 CARDINAL
                       Johannesburg GPE
                                  3 CARDINAL
                      February 2022 DATE
                         2022/03/05 CARDINAL
                    Sophie Williams PERSON
                                082 CARDINAL
                           221 1549 DATE
                         Conditions PERSON
                           3 months DATE


In [409]:
ruler = nlp.add_pipe('entity_ruler' , before = 'ner')

In [410]:
patterns = [
    {'label': 'COMPANY' , 'pattern' : 'Brilliant Air Conditioners PTY LTD.'},
    {'label': 'DATE' , 'pattern' : '03 FEB 2022'},
    {'label': 'DATE' , 'pattern' : '15 February 2022'},
    {'label': 'DATE' , 'pattern' : '2022/03/05'},
    {'label': 'EMAIL' , 'pattern' : 'sophiew@brilliantac.com'},
    {'label': 'PHONE' , 'pattern' : '082 221 1549'},
    
]


In [411]:
ruler.add_patterns(patterns)

In [412]:
doc  = nlp(text)

In [413]:
for entity in doc.ents:
    print(f"{entity.text:>35} {entity.label_}")

Brilliant Air Conditioners PTY LTD. COMPANY
                                 03 CARDINAL
                           Feb 2022 DATE
                                  1 CARDINAL
                           John Doe PERSON
                                  2 CARDINAL
                       Johannesburg GPE
                                  3 CARDINAL
                   15 February 2022 DATE
                         2022/03/05 DATE
                    Sophie Williams PERSON
                       082 221 1549 PHONE
            sophiew@brilliantac.com EMAIL
                         Conditions PERSON
                           3 months DATE


In [414]:
from spacy.matcher import Matcher

In [415]:
matcher = Matcher(nlp.vocab)

In [416]:
pattern1 = [
    {"IS_DIGIT":True},
    {"IS_ALPHA":True},
    {"IS_DIGIT":True},
    {"IS_ALPHA":True},
    {"LENGTH":10 , 'OP':'+'}
]



matcher.add("dates_between", [pattern1], greedy='LONGEST')



In [417]:

pattern2 = [
    {"IS_DIGIT":True},
    {"IS_ALPHA":True},
    {"IS_DIGIT":True},
    {"ORTH":"."}
]


matcher.add("document_date", [pattern2])



In [418]:
pattern3 = [
    {"POS":'VERB'},
    {"IS_ALPHA":True},
    {"IS_ALPHA":True},
    {"POS":'ADP'},
    {"IS_DIGIT":True},
    {"IS_DIGIT":True},
    {"IS_DIGIT":True},
    {"ORTH":"."}
]


matcher.add("contact_person", [pattern3])



In [419]:
pattern4 = [
    {"LIKE_EMAIL":True},
]

matcher.add("contact_email", [pattern4])


In [420]:
lemmas = ['guarantee']


pattern5 = [
    {"POS":'VERB', 'LEMMA':{'IN' : lemmas}},
    {"POS":'ADP'},
    {"IS_DIGIT":True}

]


matcher.add("guaranteed", [pattern5] )



In [425]:
pattern6 = [
    {"IS_DIGIT":True},

    {"IS_DIGIT":True},

    {"IS_DIGIT":True}

]


matcher.add("contact_number", [pattern6])


In [429]:
matches = matcher(doc)

In [430]:
matches

[(6813683823940948672, 11, 15),
 (5464600899058834445, 94, 97),
 (14240950106243003388, 90, 98),
 (1156907039114809599, 101, 102),
 (13263265796178019049, 151, 154),
 (13767607165787426483, 75, 80)]

In [431]:
for match in matches:
    print(match, doc[match[1]:match[2]])

(6813683823940948672, 11, 15) 03 Feb 2022.
(5464600899058834445, 94, 97) 082 221 1549
(14240950106243003388, 90, 98) contact Sophie Williams on 082 221 1549.
(1156907039114809599, 101, 102) sophiew@brilliantac.com
(13263265796178019049, 151, 154) guaranteed for 3
(13767607165787426483, 75, 80) 15 February 2022 and 2022/03/05


In [432]:
for i in range(0 , len(matches)):
    index = matches[i]
    tag = nlp.vocab[matches[i][0]].text
    text2 = doc[matches[i][1]:matches[i][2]]
    
    print(f'{tag}: {text2}')

document_date: 03 Feb 2022.
contact_number: 082 221 1549
contact_person: contact Sophie Williams on 082 221 1549.
contact_email: sophiew@brilliantac.com
guaranteed: guaranteed for 3
dates_between: 15 February 2022 and 2022/03/05


In [445]:
for entity in doc.ents:
    if entity.label_ == 'COMPANY':
        print(f'company_name:{entity.text}')
        
    if entity.label_ == 'GPE':
        print(f'locatoin:{entity.text}') 
        

for i in range(0 , len(matches)):
    index = matches[i]
    tag = nlp.vocab[matches[i][0]].text
    text2 = doc[matches[i][1]:matches[i][2]]
    
    print(f'{tag}: {text2}')
                

company_name:Brilliant Air Conditioners PTY LTD.
locatoin:Johannesburg
document_date: 03 Feb 2022.
contact_number: 082 221 1549
contact_person: contact Sophie Williams on 082 221 1549.
contact_email: sophiew@brilliantac.com
guaranteed: guaranteed for 3
dates_between: 15 February 2022 and 2022/03/05


In [442]:
for entity in doc.ents:
    if entity.label_ == 'COMPANY':
        company_name = entity.text
    if entity.label_ == 'GPE':
        location = entity.text


In [456]:
for match in matches[0:10]:
    print(nlp.vocab[match[0]].text)
    

document_date
contact_number
contact_person
contact_email
guaranteed
dates_between


In [483]:
for match in matches[0:10]:
    if nlp.vocab[match[0]].text == 'document_date':
        document_date= doc[match[1]:match[2]]
        
    if nlp.vocab[match[0]].text == 'contact_number':
        contact_number= doc[match[1]:match[2]]
        
    if nlp.vocab[match[0]].text == 'contact_person':
        contact_person= doc[match[1]:match[2]][1:3]
        
    if nlp.vocab[match[0]].text == 'contact_email':
        contact_email= doc[match[1]:match[2]]
        
    if nlp.vocab[match[0]].text == 'guaranteed':
        guaranteed= doc[match[1]:match[2]][2:3]
        
    if nlp.vocab[match[0]].text == 'dates_between':
        dates_between= doc[match[1]:match[2]]

In [484]:
print(f"company_name: {company_name}")
print(f"document_date: {document_date}")
print(f"location: {location}")
print(f"dates_between: {dates_between}")
print(f"contact_person: {contact_person}")
print(f"contact_email: {contact_email}")
print(f"contact_number: {contact_number}")
print(f"guaranteed: {guaranteed}")

company_name: Brilliant Air Conditioners PTY LTD.
document_date: 03 Feb 2022.
location: Johannesburg
dates_between: 15 February 2022 and 2022/03/05
contact_person: Sophie Williams
contact_email: sophiew@brilliantac.com
contact_number: 082 221 1549
guaranteed: 3
